In [1]:
pip install telethon python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for telethon: filename=Telethon-1.36.0-py3-none-any.whl size=680881 sha256=3933be68cb57c94b5cd632873b7157b88174cc609934f3bb86c1ac14ce539397
  Stored in directory: /root/.cache/pip/wheels/4f/6f/86/da2c68242ca7dd92786186731ee763b7ecac8866666ac47170
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=5a01a0ebf3be7391c460c749d622926882db640087c45aa69c3d516eb92a92e7
  Stored in dir

In [2]:
from telethon import TelegramClient, events
from telegram.ext import Application, CommandHandler, CallbackQueryHandler, MessageHandler, filters
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
import asyncio
import nest_asyncio

# Применяем патч для поддержки asyncio в Jupyter
nest_asyncio.apply()

# Настройки Telethon
api_id = '25127378'
api_hash = 'af94b0a0e93cf294ba4e9adf1e9a8dc3'
target_channel = 'Cr1000000'

# Настройки Telegram Bot
bot_token = '7531383363:AAFZW32Ks8NAxuif-iRko62KpZ68JWvfotM'

# Инициализация клиента Telethon
client = TelegramClient('session_name', api_id, api_hash)

# Переменные для отслеживания статуса работы бота и списка каналов
is_running = False
source_channels = set()  # Используем множество для хранения уникальных каналов

async def start_client():
    await client.start()
    print("Telethon client started")

# Функция для пересылки новых сообщений
async def forward_message(event):
    message = event.message
    if message.text:  # Пересылаем только текстовые сообщения
        await client.send_message(target_channel, message.text)

# Обработчик команды /start для бота
async def start(update: Update, context):
    keyboard = [
        [InlineKeyboardButton("Запустить", callback_data='start')],
        [InlineKeyboardButton("Остановить", callback_data='stop')],
        [InlineKeyboardButton("Добавить канал", callback_data='add_channel')],
        [InlineKeyboardButton("Удалить канал", callback_data='remove_channel')],
        [InlineKeyboardButton("Список каналов", callback_data='list_channels')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text("Выберите действие:", reply_markup=reply_markup)

# Обработчик текстовых сообщений
async def handle_text(update: Update, context):
    if 'action' in context.user_data:
        action = context.user_data['action']

        if action == 'add_channel':
            channel = update.message.text
            if channel not in source_channels:
                source_channels.add(channel)
                await update.message.reply_text(f"Канал {channel} добавлен в список источников.")
                if is_running:
                    client.add_event_handler(forward_message, events.NewMessage(chats=channel))
            else:
                await update.message.reply_text(f"Канал {channel} уже есть в списке источников.")

            del context.user_data['action']  # Сбрасываем действие после обработки

        elif action == 'remove_channel':
            channel = update.message.text
            if channel in source_channels:
                source_channels.remove(channel)
                await update.message.reply_text(f"Канал {channel} удален из списка источников.")
                if is_running:
                    client.remove_event_handler(forward_message, events.NewMessage(chats=channel))
            else:
                await update.message.reply_text(f"Канал {channel} не найден в списке источников.")

            del context.user_data['action']  # Сбрасываем действие после обработки

# Обработчик нажатий на кнопки
async def button(update: Update, context):
    query = update.callback_query
    await query.answer()

    if query.data == 'start':
        await handle_start(query)
    elif query.data == 'stop':
        await handle_stop(query)
    elif query.data == 'add_channel':
        await query.message.reply_text('Введите канал для добавления (например, @channel_username):')
        context.user_data['action'] = 'add_channel'
    elif query.data == 'remove_channel':
        await query.message.reply_text('Введите канал для удаления (например, @channel_username):')
        context.user_data['action'] = 'remove_channel'
    elif query.data == 'list_channels':
        await handle_list_channels(query)

async def handle_start(query):
    global is_running
    if not is_running:
        await query.message.reply_text("Бот запущен и начнет копировать новые сообщения.")
        await start_client()

        for source_channel in source_channels:
            client.add_event_handler(forward_message, events.NewMessage(chats=source_channel))

        is_running = True
    else:
        await query.message.reply_text("Бот уже запущен.")

async def handle_stop(query):
    global is_running
    if is_running:
        await query.message.reply_text("Бот остановлен.")
        for source_channel in source_channels:
            client.remove_event_handler(forward_message, events.NewMessage(chats=source_channel))

        await client.disconnect()
        is_running = False
    else:
        await query.message.reply_text("Бот уже остановлен.")

# async def add_channel(update, context):
#     if 'action' in context.user_data and context.user_data['action'] == 'add_channel':
#         channel = update.message.text
#         if channel not in source_channels:
#             source_channels.add(channel)
#             await update.message.reply_text(f"Канал {channel} добавлен в список источников.")
#             if is_running:
#                 client.add_event_handler(forward_message, events.NewMessage(chats=channel))
#         else:
#             await update.message.reply_text(f"Канал {channel} уже есть в списке источников.")
#         del context.user_data['action']

# async def remove_channel(update, context):
#     if 'action' in context.user_data and context.user_data['action'] == 'remove_channel':
#         channel = update.message.text
#         if channel in source_channels:
#             source_channels.remove(channel)
#             await update.message.reply_text(f"Канал {channel} удален из списка источников.")
#             if is_running:
#                 client.remove_event_handler(forward_message, events.NewMessage(chats=channel))
#         else:
#             await update.message.reply_text(f"Канал {channel} не найден в списке источников.")
#         del context.user_data['action']

async def handle_list_channels(query):
    if source_channels:
        channels_list = "\n".join(source_channels)
        await query.message.reply_text(f"Текущие каналы-источники:\n{channels_list}")
    else:
        await query.message.reply_text("Список каналов-источников пуст.")

# Главная функция для инициализации и запуска бота
# async def main():
#     application = Application.builder().token(bot_token).build()

#     # Добавляем обработчики
#     application.add_handler(CommandHandler('start', start))
#     application.add_handler(CallbackQueryHandler(button_handler))
#     application.add_handler(CommandHandler('add_channel', add_channel))
#     application.add_handler(CommandHandler('remove_channel', remove_channel))

#     # Запуск бота
#     await application.run_polling()

# if __name__ == "__main__":
#     asyncio.run(main())

async def main():
    application = Application.builder().token(bot_token).build()

    start_handler = CommandHandler('start', start)
    button_handler = CallbackQueryHandler(button)
    text_handler = MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text)

    application.add_handler(start_handler)
    application.add_handler(button_handler)
    application.add_handler(text_handler)

    await application.run_polling()

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

Please enter your phone (or bot token): +79094447862
Please enter the code you received: 83670
Please enter your password: ··········
Signed in successfully as Sergey; remember to not break the ToS or you will risk an account ban!
Telethon client started


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/_application.py", line 1335, in process_update
    await coroutine
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/_handlers/basehandler.py", line 157, in handle_update
    return await self.callback(update, context)
  File "<ipython-input-2-e91edd8a2a90>", line 49, in handle_text
    if 'action' in context.user_data:
TypeError: argument of type 'NoneType' is not iterable
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/_application.py", line 1335, in process_update
    await coroutine
  File "/usr/local/lib/python3.10/dist-packages/telegram/ext/_handlers/basehandler.py", line 157, in handle_update
    return await self.callback(update, context)
  File "<ipython-input

RuntimeError: Cannot close a running event loop